In [1]:
import csv
import datetime
import dateutil

In [2]:
def parse_date(s):
    return dateutil.parser.parse(s)

assert str(parse_date("2018-09-01T23:00:00-07:00")) == "2018-09-01 23:00:00-07:00"

In [3]:
def read_intervals(filename):
    records = []
    with open(filename) as file:
        for record in csv.DictReader(file):
            begin = dateutil.parser.parse(record["timestamp"])
            duration = datetime.timedelta(milliseconds = int(record["duration"]))
            records.append((begin, begin + duration))
    return records

intervals = read_intervals("sleep-regular.csv")
assert len(intervals) == 15
assert intervals[0][0] == parse_date("2018-09-01T23:00:00-07:00")
assert intervals[0][1] == parse_date("2018-09-02T07:00:00-07:00")

In [4]:
def contains(intervals, t):
    for interval in intervals:
        if interval[0] > t:
            break
        if t < interval[1]:
            return True
    return False

assert not contains([], 5)
assert not contains([(1, 5), (6, 10)], 5)
assert contains([(1, 10)], 5)
assert contains([(1, 2), (5, 10)], 5)

In [5]:
def compare_epochs(intervals):
    one_minute = datetime.timedelta(minutes = 1)
    one_day = datetime.timedelta(days = 1)
    epochs = []
    begin = intervals[0][0]
    end = intervals[-1][1] - one_day
    while begin < end:
        epochs.append(0 if contains(intervals, begin) ^ contains(intervals, begin + one_day) else 1)
        begin += one_minute
    return epochs

epochs = compare_epochs([ 
    (parse_date("2018-09-01T22:00:00-07:00"), parse_date("2018-09-02T06:00:00-07:00")),
    (parse_date("2018-09-02T23:00:00-07:00"), parse_date("2018-09-03T07:00:00-07:00")),
    (parse_date("2018-09-03T22:00:00-07:00"), parse_date("2018-09-04T06:00:00-07:00")) 
])
assert len(epochs) == 32 * 60
assert sum(epochs) == 29 * 60

In [6]:
def calculate_sri(epochs):
    return int(100  * sum(epochs) / len(epochs))

assert calculate_sri([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) == 100
assert calculate_sri([0, 0, 0, 0, 0, 1, 1, 1, 1, 1]) == 50
assert calculate_sri([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 0

In [7]:
def calculate_sri_from_file(filename):
    intervals = read_intervals(filename)
    epochs = compare_epochs(intervals)
    return calculate_sri(epochs)

assert calculate_sri_from_file("sleep-regular.csv") == 100
assert calculate_sri_from_file("sleep-alternate.csv") == 0